In [2]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OrdinalEncoder, LabelEncoder,OneHotEncoder
import haversine as hs
from haversine import Unit
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge,Lasso,ElasticNet
from sklearn.ensemble import RandomForestRegressor 
from sklearn.metrics import mean_squared_error

In [3]:
df1=pd.read_csv('https://raw.githubusercontent.com/ssharma2323/Swati_Assignment/main/FinalTrain.csv')
df1.head()

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,Time_Order_picked,Weather_conditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken (min)
0,0xcdcd,DEHRES17DEL01,36.0,4.2,30.327968,78.046106,30.397968,78.116106,12-02-2022,21:55,22:10,Fog,Jam,2,Snack,motorcycle,3.0,No,Metropolitian,46
1,0xd987,KOCRES16DEL01,21.0,4.7,10.003064,76.307589,10.043064,76.347589,13-02-2022,14:55,15:05,Stormy,High,1,Meal,motorcycle,1.0,No,Metropolitian,23
2,0x2784,PUNERES13DEL03,23.0,4.7,18.562450,73.916619,18.652450,74.006619,04-03-2022,17:30,17:40,Sandstorms,Medium,1,Drinks,scooter,1.0,No,Metropolitian,21
3,0xc8b6,LUDHRES15DEL02,34.0,4.3,30.899584,75.809346,30.919584,75.829346,13-02-2022,09:20,09:30,Sandstorms,Low,0,Buffet,motorcycle,0.0,No,Metropolitian,20
4,0xdb64,KNPRES14DEL02,24.0,4.7,26.463504,80.372929,26.593504,80.502929,14-02-2022,19:50,20:05,Fog,Jam,1,Snack,scooter,1.0,No,Metropolitian,41


In [4]:
import pandas as pd
import math

def calculate_distance(row):
    try: 
        lat1 = row['Restaurant_latitude']
        lon1 = row['Restaurant_longitude']
        lat2 = row['Delivery_location_latitude']
        lon2 = row['Delivery_location_longitude']
        
        '''
        Since for few locations where lat long is negative, Restaurant location was of sea which is not possible and if we make it 
        positive then it is at feasible distance to delivery location, hence making the adjustment accordingly
        '''
        if lat1<0:
            lat1=lat1*(-1)
        if lon1<0:
            lon1=lon1*(-1)
        # Convert degrees to radians
        lat1_rad = math.radians(lat1)
        lon1_rad = math.radians(lon1)
        lat2_rad = math.radians(lat2)
        lon2_rad = math.radians(lon2)

        # Radius of the Earth in kilometers
        radius = 6371

        # Calculate the differences between the coordinates
        dlat = lat2_rad - lat1_rad
        dlon = lon2_rad - lon1_rad

        # Haversine formula
        a = math.sin(dlat/2) * math.sin(dlat/2) + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(dlon/2) * math.sin(dlon/2)
        c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
        distance = radius * c

        return distance
    except (TypeError, ValueError):
        return None

# Calculate distances and assign them to a new column
df1['distance'] = df1.apply(calculate_distance, axis=1)


In [5]:
df1.head()

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,...,Weather_conditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken (min),distance
0,0xcdcd,DEHRES17DEL01,36.0,4.2,30.327968,78.046106,30.397968,78.116106,12-02-2022,21:55,...,Fog,Jam,2,Snack,motorcycle,3.0,No,Metropolitian,46,10.280582
1,0xd987,KOCRES16DEL01,21.0,4.7,10.003064,76.307589,10.043064,76.347589,13-02-2022,14:55,...,Stormy,High,1,Meal,motorcycle,1.0,No,Metropolitian,23,6.242319
2,0x2784,PUNERES13DEL03,23.0,4.7,18.562450,73.916619,18.652450,74.006619,04-03-2022,17:30,...,Sandstorms,Medium,1,Drinks,scooter,1.0,No,Metropolitian,21,13.787860
3,0xc8b6,LUDHRES15DEL02,34.0,4.3,30.899584,75.809346,30.919584,75.829346,13-02-2022,09:20,...,Sandstorms,Low,0,Buffet,motorcycle,0.0,No,Metropolitian,20,2.930258
4,0xdb64,KNPRES14DEL02,24.0,4.7,26.463504,80.372929,26.593504,80.502929,14-02-2022,19:50,...,Fog,Jam,1,Snack,scooter,1.0,No,Metropolitian,41,19.396618


In [6]:
x = df1.drop(['Time_taken (min)'], axis=1)
y = df1['Time_taken (min)']

In [7]:
class Imputer(BaseEstimator, TransformerMixin):
    def fit(self, df):
        return self
    def transform(self, df):
        df['Delivery_person_Ratings'] = df['Delivery_person_Ratings'].fillna(df['Delivery_person_Ratings'].median())
        df['Delivery_person_Age'] = df['Delivery_person_Age'].fillna(df['Delivery_person_Age'].mean())
        df['multiple_deliveries'] = df['multiple_deliveries'].fillna(df['multiple_deliveries'].mode()[0])
        df['Weather_conditions'] = df['Weather_conditions'].fillna(df['Weather_conditions'].mode()[0])
        df['Road_traffic_density'] = df['Road_traffic_density'].fillna(df['Road_traffic_density'].mode()[0])
        df['Festival'] = df['Festival'].fillna(df['Festival'].mode()[0])
        df['City'] = df['City'].fillna(df['City'].mode()[0])
        return df



class drop_feature(BaseEstimator, TransformerMixin):
    def __init__(self, columns_to_drop):
        self.columns_to_drop = columns_to_drop
    def fit(self, df):
        return self
    def transform(self, df):
        for i in columns_to_drop:
            df.drop(i, axis=1, inplace=True)
        return df


class OrdinalEncoding(BaseEstimator, TransformerMixin):
    def __init__(self, column_mappings):
        self.column_mappings = column_mappings
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        for col, mapping in self.column_mappings.items():
            X[col] = X[col].map(mapping)
        return X

class OneHotEncoding(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        self.columns = columns
        self.encoder = None
    
    def fit(self, X, y=None):
        self.encoder = OneHotEncoder(sparse=False, drop='first').fit(X[self.columns])
        return self
    
    def transform(self, X, y=None):
        encoded = self.encoder.transform(X[self.columns])
        # Convert boolean values to integers (1 and 0)
        encoded = encoded.astype(int)
        
        # Create new column names
        new_columns = self.encoder.get_feature_names_out(self.columns)
        
        # Create a new DataFrame with the encoded features
        encoded_df = pd.DataFrame(encoded, columns=new_columns, index=X.index)
        
        # Drop the original columns from the input DataFrame
        X.drop(self.columns, axis=1, inplace=True)
        
        # Concatenate the original DataFrame with the encoded DataFrame
        X_encoded = pd.concat([X, encoded_df], axis=1)
        
        return X_encoded

class FeatureScaling(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        self.columns = columns
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X_scaled = X.copy()
        scaler = StandardScaler()
        X_scaled[self.columns] = scaler.fit_transform(X[self.columns])
        return X_scaled


In [8]:
from sklearn.pipeline import Pipeline

# Define the column mappings
column_mappings = {
    'Road_traffic_density': {'Jam': 4, 'High': 3, 'Medium': 2, 'Low': 1},
    'Festival': {'No': 0, 'Yes': 1},
    'City': {'Metropolitian': 2, 'Urban': 1, 'Semi-Urban': 3}
}

columns_to_drop=['ID','Delivery_person_ID','Restaurant_latitude','Restaurant_longitude','Delivery_location_latitude','Delivery_location_longitude','Order_Date','Time_Orderd','Time_Order_picked','Type_of_order']

# Define the pipeline
pipe = Pipeline([
    ('Imputer',Imputer()),
    ('drop_feature',drop_feature(columns_to_drop)),
    ('ordinal_encoding', OrdinalEncoding(column_mappings)),
    ('one_hot_encoding', OneHotEncoding(columns=['Weather_conditions', 'Type_of_vehicle'])),
    ('feature_scaling', FeatureScaling(columns=['Delivery_person_Age', 'Delivery_person_Ratings', 'distance']))
])



In [9]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=10)

In [10]:
x_train=pipe.fit_transform(x_train)
x_test=pipe.transform(x_test)

/config/.local/lib/python3.8/site-packages/sklearn/preprocessing/_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [11]:
x_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 31908 entries, 44352 to 17673
Data columns (total 16 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Delivery_person_Age               31908 non-null  float64
 1   Delivery_person_Ratings           31908 non-null  float64
 2   Road_traffic_density              31908 non-null  int64  
 3   Vehicle_condition                 31908 non-null  int64  
 4   multiple_deliveries               31908 non-null  float64
 5   Festival                          31908 non-null  int64  
 6   City                              31908 non-null  int64  
 7   distance                          31908 non-null  float64
 8   Weather_conditions_Fog            31908 non-null  int64  
 9   Weather_conditions_Sandstorms     31908 non-null  int64  
 10  Weather_conditions_Stormy         31908 non-null  int64  
 11  Weather_conditions_Sunny          31908 non-null  int64  
 12  Weath

In [12]:
x_test

,Delivery_person_Age,Delivery_person_Ratings,Road_traffic_density,Vehicle_condition,multiple_deliveries,Festival,City,distance,Weather_conditions_Fog,Weather_conditions_Sandstorms,Weather_conditions_Stormy,Weather_conditions_Sunny,Weather_conditions_Windy,Type_of_vehicle_electric_scooter,Type_of_vehicle_motorcycle,Type_of_vehicle_scooter
35338,-0.802265,-0.117990,1,0,0.0,0,2,-0.074139,0,0,0,0,0,0,1,0
6939,-1.507292,-0.737667,2,0,1.0,0,1,0.721514,0,1,0,0,0,0,1,0
2810,1.312818,-1.047505,1,1,1.0,0,2,0.728860,0,0,0,0,0,0,1,0
42524,-1.507292,-0.427829,2,2,1.0,0,2,-0.905700,0,0,0,0,1,0,0,1
6913,-1.331035,0.191848,4,2,1.0,0,1,-0.391509,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24331,-0.449751,0.811524,1,0,1.0,0,2,1.824626,0,0,0,1,0,0,1,0
15361,0.431534,1.121363,2,1,1.0,0,2,-0.927855,0,0,1,0,0,0,1,0
40353,-0.273494,0.501686,1,2,0.0,0,2,-1.198966,1,0,0,0,0,1,0,0
25544,-1.154778,0.811524,1,2,1.0,0,1,-1.465090,0,1,0,0,0,0,0,1


In [13]:
# Apply the pipeline to the DataFrame
#df2 = pipe.fit_transform(df1)

In [14]:
models = {
    "LinearRegression": LinearRegression(n_jobs=-1),
    "Ridge": Ridge(),
    "Lasso": Lasso(),
    "ElasticNet": ElasticNet(),
    "RandomForest": RandomForestRegressor()
}

In [15]:
from sklearn.metrics import r2_score

def train_model(model_name,model,x_train=x_train,x_test=x_test,y_train=y_train,y_test=y_test):
    print('*'*30)
    model.fit(x_train,y_train)
    y_pred=model.predict(x_test)
    print(model_name,':')
    r2=r2_score(y_test,y_pred)
    print('r2_score',r2)
    n_samples=len(y_test)
    adjusted_r2 = 1 - ((1 - r2) * (n_samples - 1)) / (n_samples - x_train.shape[1] - 1)
    print('adjusted_r2_score',adjusted_r2)   
    print("MSE", ":", mean_squared_error(y_pred, y_test))
    print("RMSE", ":", np.sqrt(mean_squared_error(y_pred, y_test)))

In [16]:
for model_name, model in models.items():
    train_model(model_name,model)

******************************
LinearRegression :
r2_score 0.5774641776111316
adjusted_r2_score 0.5769692238694065
MSE : 37.91969351771457
RMSE : 6.157896842081278
******************************
Ridge :
r2_score 0.577447155146761
adjusted_r2_score 0.5769521814651114
MSE : 37.92122116720047
RMSE : 6.158020880705137
******************************
Lasso :
r2_score 0.41077184801350186
adjusted_r2_score 0.41008163273919307
MSE : 52.879187400055045
RMSE : 7.271807711982973
******************************
ElasticNet :
r2_score 0.39754798738633257
adjusted_r2_score 0.3968422818294237
MSE : 54.065938239944174
RMSE : 7.352954388539628
******************************
RandomForest :
r2_score 0.8065309022809323
adjusted_r2_score 0.8063042747413243
MSE : 17.362525262778977
RMSE : 4.166836361411254
